In [ ]:
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

sys.path.append(str(Path("__file__").resolve().parent.parent.parent))

from sub import pdr, threeDimensional

# データ読み込み
acc_df = pd.read_csv("../../../dataset/ThreeDimensional/Accelerometer.csv")
gyro_df = pd.read_csv("../../../dataset/ThreeDimensional/Gyroscope.csv")
baro_df = pd.read_csv("../../../dataset/ThreeDimensional/Barometer.csv")

# 角度と軌跡の推定
angle_df, trajectory_df = pdr.estimate_trajectory(acc_df, gyro_df)
# 安定区間の探索
stable_intervals = threeDimensional.find_stable_intervals(baro_df, "X (hPa)", "ts")
# プロット準備
fig, ax = plt.subplots()
# other points
ax.scatter(
    trajectory_df["x_displacement"],
    -trajectory_df["y_displacement"],
    color="blue",
    s=3,
)

# 軸のアスペクト比設定
ax.set_aspect("equal")
# プロット表示
plt.show()

In [ ]:
from sub import drift

ground_truth_point_df = pd.DataFrame(
    {"ts": [0, 69], "x": [0, 0], "y": [0, 0]},
)


remove_drift_angle_df, remove_drift_trajectory = drift.remove_drift_in_angle_df(
    acc_df,
    angle_df,
    ground_truth_point_df,
)

In [ ]:
import importlib

importlib.reload(threeDimensional)
_, _, _, _, baro_df_stable_time_list = threeDimensional.process_baro_data(
    baro_df,
    stable_intervals,
)

In [ ]:
# プロット準備
fig, ax = plt.subplots()

# other points
ax.scatter(
    remove_drift_trajectory["x_displacement"],
    -remove_drift_trajectory["y_displacement"],
    color="blue",
    s=3,
)

for index, stable_interval in enumerate(baro_df_stable_time_list):
    if index == 0:
        threeDimensional.highlight_stable_intervals(
            stable_interval,
            "red",
            ax,
            remove_drift_trajectory,
        )
    if index == 1:
        threeDimensional.highlight_stable_intervals(
            stable_interval,
            "green",
            ax,
            remove_drift_trajectory
        )


# 軸のアスペクト比設定
ax.set_aspect("equal")

# プロット表示
plt.show()

In [ ]:
x = baro_df["ts"]
y = baro_df["X (hPa)"]


plt.plot(
    x,
    y,
)

plt.show()

In [ ]:
# 安定区間を見つけるための関数
def find_stable_intervals(df, pressure_col, time_col, threshold=0.03, duration=10):
    stable_intervals = []
    start_index = 0
    window_size = int(duration)  # duration秒に相当するデータポイント数
    is_stable = False

    while start_index + window_size < len(df):
        window = df.iloc[start_index : start_index + window_size]
        if (window[pressure_col].max() - window[pressure_col].min()) <= threshold:
            stable_intervals.append(
                (window[time_col].iloc[0], window[time_col].iloc[-1]),
            )
            is_stable = True

        start_index += 1

    return stable_intervals


# 安定区間の探索
stable_intervals = find_stable_intervals(baro_df, "X (hPa)", "ts")


# プロット
plt.plot(x, y)
for start, end in stable_intervals:
    plt.axvspan(start, end, color="green", alpha=0.3)

plt.xlabel("ts")
plt.ylabel("Pressure (hPa)")
plt.show()

In [ ]:
import importlib

from sub import drift

importlib.reload(drift)


ground_truth_point_df = pd.DataFrame(
    {"ts": [0, 69], "x": [0, 0], "y": [0, 0]},
)


remove_drift_angle_df, remove_drift_trajectory = drift.remove_drift_in_angle_df(
    acc_df,
    angle_df,
    ground_truth_point_df,
)

fig, ax = plt.subplots()

plt.colorbar(
    ax.scatter(
        remove_drift_trajectory["x_displacement"],
        -remove_drift_trajectory["y_displacement"],
        c=remove_drift_trajectory["ts"],
        cmap="rainbow",
        s=3,
    ),
)
ax.set_aspect("equal")

plt.show()